# Automation of Evaluation of codes

This code tries to automate evaluation of assignments on a very basic level.
Students submit assignment with filename of format 
L_Labnumber_FirstName_EntryNumber_CourseCode_Year.c/cpp.
Some input/output files are stored on system.
This code run their codes on these input files 
grades them on the basis of number of testcases passed.

There is a folder in current working directory with name L0+"lab_number".\
Within this folder, there are input and output files with name l0+"lab_number"+"input"/"output"+"file_number".\
Along with these input output files, there is a folder L0+"lab_number"+"_codes" containing all code files submitted by students


In [1]:
import os
import shutil
import pandas as pd
print(os.name)
#nt is for windows
#This code is designed to work for windows

nt


In [2]:
curr_folder=os.getcwd()

In [3]:
n=int(input())#lab_number

2


In [4]:
lab_folder = "L0"+str(n)
print(lab_folder)

L02


In [5]:
code_folder = lab_folder+"_codes"
print(code_folder)

L02_codes


In [6]:
path_to_lab_folder = os.path.join(curr_folder,lab_folder)

In [7]:
relative_path_to_code_folder=os.path.join(lab_folder,code_folder)

In [8]:
abs_path_to_code_folder=os.path.join(path_to_lab_folder,code_folder)

In [9]:
code_list=os.listdir(relative_path_to_code_folder)
print(code_list)

['L02_Ambuj_2018CSB1125_CS201_2020.c', 'L02_Anuj_2017CSB1125_CS201_2020.c', 'L02_Diksha_2019MCB1216_CS201_2020.c', 'L02_Disha_2018CSB1216_CS201_2020.c', 'L02_Tanay_2018CSB1124_CS201_2020.c', 'L02_Tanmay_2019CSB1124_CS201_2020.c', 'L02_Tanuj_2019CSB1125_CS201_2020.c', 'L02_Tarun_2017CSB1123_CS201_2020.c']


In [10]:
file_list = os.listdir(lab_folder)

In [11]:
input_files=[f for f in file_list if f[0:8]=='l0'+str(n)+"input"]
output_files=[f for f in file_list if f[0:9]=='l0'+str(n)+"output"]
print(input_files)
print(output_files)

['l02input1.txt', 'l02input2.txt', 'l02input3.txt', 'l02input4.txt', 'l02input5.txt', 'l02input6.txt']
['l02output1.txt', 'l02output2.txt', 'l02output3.txt', 'l02output4.txt', 'l02output5.txt', 'l02output6.txt']


In [12]:
record={'Entry_Number':[],'Name':[],'Compilation_Status':[],'Score':[],'Remark':[]}

In [13]:
num_codes=len(code_list)

In [14]:
os.chdir(path_to_lab_folder)

In [15]:
for i in range(num_codes):
    file_name=code_list[i]
    print("Evaluating",file_name)
    info = file_name.split('_')
    entry_number=info[2]
    name=info[1]
    record['Entry_Number'].append(entry_number)
    record['Name'].append(name)
    record['Score'].append(0)
    record['Remark'].append('')
    src=os.path.join(abs_path_to_code_folder,file_name)
    destination=os.path.join(path_to_lab_folder,file_name)
    shutil.copyfile(src,destination)
    if file_name[-2:]=='.c':
        compilation_status = os.system("gcc "+file_name+" -o "+entry_number)
    else:
        compilation_status = os.system("g++ "+file_name+" -o "+entry_number)
    if compilation_status==0:
        record['Compilation_Status'].append('Success')
        test_cases_failed=[]
        num_test_cases=len(input_files)
        for j in range(num_test_cases):
            os.system(entry_number+" < "+input_files[j]+" > "+entry_number+str(j+1)+".txt")
            diff = os.system("FC "+output_files[j]+" "+entry_number+str(j+1)+".txt")
            if(diff==1):
                test_cases_failed.append(j+1)
            os.remove(os.path.join(path_to_lab_folder,entry_number+str(j+1)+".txt"))
        num_cases_failed=len(test_cases_failed)
        if num_cases_failed==0:
            record['Remark'][i]='All test cases passed'
            record['Score'][i]=100
        else:
            remark='failed for test cases '+str(test_cases_failed[0])
            for _ in range(1,num_cases_failed):
                remark=remark+','+str(test_cases_failed[_])
            record['Remark'][i]=remark
            record['Score'][i]=((num_test_cases-num_cases_failed)/num_test_cases)*100
        path_executable=os.path.join(path_to_lab_folder,entry_number+".exe")
        os.remove(path_executable)
    else:
        record['Compilation_Status'].append('Failed')
        record['Remark'][i]='Compilation Failed'
    os.remove(destination)

Evaluating L02_Ambuj_2018CSB1125_CS201_2020.c
Evaluating L02_Anuj_2017CSB1125_CS201_2020.c
Evaluating L02_Diksha_2019MCB1216_CS201_2020.c
Evaluating L02_Disha_2018CSB1216_CS201_2020.c
Evaluating L02_Tanay_2018CSB1124_CS201_2020.c
Evaluating L02_Tanmay_2019CSB1124_CS201_2020.c
Evaluating L02_Tanuj_2019CSB1125_CS201_2020.c
Evaluating L02_Tarun_2017CSB1123_CS201_2020.c


In [16]:
marks_record=pd.DataFrame(record)

In [17]:
print(marks_record)

  Entry_Number    Name Compilation_Status       Score  \
0  2018CSB1125   Ambuj             Failed    0.000000   
1  2017CSB1125    Anuj             Failed    0.000000   
2  2019MCB1216  Diksha            Success   83.333333   
3  2018CSB1216   Disha            Success   50.000000   
4  2018CSB1124   Tanay            Success   66.666667   
5  2019CSB1124  Tanmay            Success  100.000000   
6  2019CSB1125   Tanuj            Success  100.000000   
7  2017CSB1123   Tarun            Success   66.666667   

                        Remark  
0           Compilation Failed  
1           Compilation Failed  
2      failed for test cases 1  
3  failed for test cases 1,4,5  
4    failed for test cases 2,3  
5        All test cases passed  
6        All test cases passed  
7    failed for test cases 3,6  


In [18]:
file_destination=os.path.join(curr_folder,info[3]+" Lab"+str(n)+" marks.csv")

In [19]:
marks_record.to_csv(file_destination)